In [1]:
import cv2
import numpy as np
import time
import imutils
from joblib import load
from dataset_io import *
import timeit
from tqdm import tqdm
import psutil
from datetime import datetime

In [2]:
def pyramid(image, split_channels, initial_scale, final_scale, scale=1.5):
    original_w=image.shape[1]
    curr_scale=initial_scale
    while curr_scale>final_scale/scale:
        w = int(original_w * curr_scale)
        yield imutils.resize(image, width=w), [imutils.resize(ch, width=w) for ch in split_channels],curr_scale
        curr_scale/=scale
        
def sliding_window(image, stepSize, windowSize):
    for y in range(0, image.shape[0], stepSize):
        for x in range(0, image.shape[1], stepSize):
            yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

In [3]:
(winW, winH) = (120,24)
DS_min_winH_m=41
DS_max_winH_m=287
initial_scale=winH/DS_min_winH_m
final_scale=winH/DS_max_winH_m

clf = load('rand_forest_clf.joblib') 
clf.set_params(n_jobs=-1)

def show(img, x, y, winW, winH, positives=[]):
    clone = img.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    for p in positives:
        x,y=p
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (255, 255, 255), 2)
    cv2.imshow("Window", clone)
    cv2.waitKey(1)

def image_predict(X,IDX,rects_count):
    global glob_RES
    if len(glob_X)!=0:
        pred=[p[1] for p in clf.predict_proba(X)]
        max_pred_ids=np.argsort(pred)[-rects_count:]
        max_preds=pred[max_pred_ids]
        max_pred_IDXs=IDX[max_pred_ids]
        
        max_preds=np.array([[e] for e in max_preds])
        res=np.append(max_pred_IDXs, max_preds, 1)
        
        glob_RES=np.concatenate((glob_RES,res))
        TERAZ SPRAWDŹ CZY TO DZIAŁA I WCZYTAJ TO PRZY WYŚWIETLANIU PALET
    
def predict(filename, scene_name, label_resolution, rects):
    X=[]
    IDX=[]
    
    image=imread_resized(scene_name, filename, label_resolution)
    split_channels=read_split_channels(scene_name, filename)
    X=[]
    for resized_img, resized_split_ch, scale in pyramid(image, split_channels, initial_scale, final_scale, scale=1.15):
        resized_ch=np.dstack(resized_split_ch)
        for (x, y, window) in sliding_window(resized_img, stepSize=4, windowSize=(winW, winH)):
            # if the window does not meet our desired window size, ignore it
            if window.shape[0] != winH or window.shape[1] != winW:
                continue
            channels_window=resized_ch[y:y + winH, x:x + winW]
            features=channels_window.flatten()
            X.append(features)
            IDX.append([scene_name, filename, scale, (x,y)])
    image_predict(X,IDX, len(rects))

In [ ]:
glob_RES=[]

glob_DATETIME = datetime.now().strftime("%d-%m-%Y_%H:%M:%S")

[predict(*row) for row in tqdm([row for row in walk_dataset()])]# if row[1]=="blue_1" and row[0]=="IMG_20200229_075416.jpg"])]
global_predict()
global_persist_results()

 12%|█▏        | 254/2092 [10:13<1:22:44,  2.70s/it]

In [ ]:
def area(rect):
    ((x,y),(x2,y2))=rect
    return (x2-x)*(y2-y)

def calc_overlapping(a,b):
    dx = min(a[1][0], b[1][0]) - max(a[0][0], b[0][0])
    dy = min(a[1][1], b[1][1]) - max(a[0][1], b[0][1])
    max_area=np.max([area(a), area(b)])
    overlap_area=dx*dy
    return overlap_area/max_area if ((dx>=0) and (dy>=0)) else 0

def calc_overlappings(rects, pred_rects):
    global glob_overlappings    
    overlappings=[np.max([calc_overlapping(pred_rect,rect) for pred_rect in pred_rects]) for rect in rects]
    glob_overlappings.extend(overlappings)

def scale_many(vals, scale):
    return [int(val/scale) for val in vals]

def draw_predicted_rectangles(filename, scene_name, label_resolution, rects):
    img=imread_resized(scene_name, filename, label_resolution)
    file_res=RES[np.where((RES[:,0] == scene_name) * (RES[:,1] == filename))]
    
    n_rects=len(rects)
    max_rows=file_res[np.argsort(file_res[:,4])[-n_rects:]]
    pred_rects=[]
    rects=[add_margin(correct_rect_ratio(rect)) for rect in rects]
    for max_row in max_rows:
        [scene_name, filename, scale, (x,y), pred]=max_row
        [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
        cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
        pred_rects.append(((x,y),(x+winW_s, y+winH_s)))
    for rect in rects:
        ((x,y),(x2,y2))=rect
        cv2.rectangle(img, (x,y), (x2,y2), (255, 0, 0), 2)
        
#     for row in file_res:
#         [scene_name, filename, scale, (x,y), pred]=row
#         [x,y,winW_s,winH_s]=scale_many([x,y,winW,winH], scale)
#         #print(winH_s)
#         if pred>0.9:
#             cv2.rectangle(img, (x,y), (x + winW_s, y + winH_s), (0, 255, 0), 2)
    save_image(img, scene_name, filename, "predicted_labels")
    calc_overlappings(rects, pred_rects)

glob_overlappings=[]
[draw_predicted_rectangles(*row) for row in walk_dataset() if row[1]=="blue_1" and row[0]=="IMG_20200229_075416.jpg"]
print("Mean overlapping: ",np.mean(glob_overlappings))

In [ ]:
# window only - 1.36s
# features only - 1.47s
# classify per image - 2.82s
# classify per scale - 8.47s
# dsiplay, features only - 52.64s